In [3]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import statistics
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [4]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [5]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [6]:
#! Set these variables
work_dir_name = "pineal3"

In [7]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = drive_root / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"

Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [9]:
labels = ["pineal"]
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="flair.t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, suffix_list=["CH", "SRS", "ED", "DT"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-12-12 19:06:31.895 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
 10%|█         | 1/10 [00:00<00:05,  1.57it/s]2024-12-12 19:06:32.633 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-CH.nii.gz for Scan(subid=1033, sesid=20171117)
2024-12-12 19:06:32.722 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-CH.nii.gz for Scan(subid=1119, sesid=20161010)
 30%|███       | 3/10 [00:00<00:01,  4.34it/s]2024-12-12 19:06:32.806 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-SRS.nii.gz for Scan(subid=1259, sesid=20200803)
2024-12-12 19:06:32.851 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-SRS.nii.gz for Scan(subid=1437, sesid=20210503)
 50%|█████     | 5/10 [00:00<00:00,  6.89it/s]2024-12-12 19:06:32.915 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pineal-SRS.nii.gz for Scan(subid=1547, sesid=20220321)
2024-12-12

Calculate Dice Score for pineal

In [11]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "pineal", ["CH", "SRS", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2, seg2_val=1)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1010: 0.34
1033: 0.74
1119: 0.45
1259: 0.76
1437: 0.82
1547: 0.80
2081: 0.87
2126: 0.81
2146: 0.25
2187: 0.75
----------
Mean: 0.66
Std: 0.23
[0.33554817275747506, 0.73992673992674, 0.4519015659955257, 0.758893280632411, 0.8157894736842105, 0.7952898550724637, 0.8718562874251496, 0.8123924268502581, 0.24566929133858267, 0.7530168946098149]


Calculate Hausdorff Distance for Pineal

In [12]:
haus = []
print("Hausdorff Dist")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "pineal", ["CH", "SRS", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=1)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

Hausdorff Dist
1010: 137.25
1033: 74.76
1119: 89.11
1259: 1.41
1437: 40.12
1547: 3.61
2081: 1.00
2126: 99.70
2146: 66.99
2187: 79.03
----------
Mean: 59.30
Std: 46.55
[137.24995014314737, 74.76128641821256, 89.11425246748274, 1.4142135623730951, 40.11661944628753, 3.605551275463989, 1.0, 99.70456156879672, 66.9880264825541, 79.02784227287718]
